In [1]:
from skimage import io, transform
import os
#from 人脸标准化 import normalization
import sklearn.preprocessing
import face_recognition
from PIL import Image
import cv2
import dlib
import numpy as np
import os

In [2]:
#通过三个点计算夹角
#b为夹角位置所在的点
#默认θ=1计算弧度，θ!不等于1时计算角度
def cos_angle(a,b,c,θ = 1):
    x,y = b-a,b-c
    Lx = np.sqrt(x.dot(x))
    Ly = np.sqrt(y.dot(y))
    cos_angle = x.dot(y)/(Lx*Ly)
    # 根据条件选择是计算弧度还是角度
    if θ != 1:
        return np.arccos(cos_angle)*360/2/np.pi
    else:
        return np.arccos(cos_angle)

In [3]:
#利用人脸标记模型 shape_predictor_68_face_landmarks.dat 标记人脸节点
def trait_angle(path):
    detector = dlib.get_frontal_face_detector()
    landmark_predictor = dlib.shape_predictor(
        r'shape_predictor_68_face_landmarks/shape_predictor_68_face_landmarks.dat')
    img = cv2.imread(path)
    faces = detector(img, 1)
    feas = []  # 关键点
    if (len(faces) > 0):
        for k, d in enumerate(faces):
            cv2.rectangle(img, (d.left(), d.top()), (d.right(), d.bottom()), (255, 255, 255))
            shape = landmark_predictor(img, d)

            for i in range(68):
                num = str(shape.part(i))[1:-1].split(",")
                feas.append([int(num[0]), int(num[1])])
    if not feas:
        print("List is empty")
        print(path+"不能识别")
        return
#     print('before:')
#     print(feas)
    feas = np.array(feas)
#     print('after:')
#     print(feas)
    s_fa = feas[45, :][1] - feas[36, :][1]
    a, b, c = feas[45, :], feas[36, :], np.array(feas[45, :][0], feas[36, :][1])
    if abs(s_fa) > 5:
        if s_fa > 0 and cos_angle(a, b, c,θ=4) >35:
            return cos_angle(a, b, c,θ=4)
        elif s_fa < 0 and cos_angle(a, b, c,θ=4) >35:
            return 360-cos_angle(a, b, c, θ=4)
        else:
            return 0
    else:
        return 0

In [4]:
def normalization(input,output):
    path =input
    out_path = output

    # 读取图片并识别人脸
    img = face_recognition.load_image_file(path)
    #这里如果识别不出人脸 那么len是不会大于0的 要检查好后再处理 不然报错
    encodings = face_recognition.face_encodings(img)
    if len(encodings) > 0:
        biden_encoding = encodings[0]
        face_locations = tuple(list(face_recognition.face_locations(img)[0]))
    else:
        print(path + ' 不能识别')
        return

    # 重新确定切割位置并切割
    top = face_locations[0]
    right = face_locations[1]
    bottom = face_locations[2]
    left = face_locations[3]
    cutting_position = (left, top, right, bottom)
    # 切割出人脸
    im = Image.open(path)

    region = im.crop(cutting_position)

    # 人脸缩放
    a = 224  # 人脸方格大小
    if region.size[0] >= a or region.size[1] >= a:
        region.thumbnail((a, a), Image.ANTIALIAS)
    else:
        region = region.resize((a, a), Image.ANTIALIAS)
    # 人脸旋转
    θ =trait_angle(path)
    # region = region.rotate(θ)
    # 保存人脸
    region.save(out_path)

In [5]:
#读取文件批量处理
#lfw 数据集处理
def read_img(path_lfw):
    map_path = []
    map_relative = []
    sonOfPathLfw = [y for y in os.listdir(path_lfw)]
    for i in sonOfPathLfw:
        path = path_lfw + i + '/'
        for x in os.listdir(path):
            if os.path.isfile(path + x):
                son_map_path = path + x
                map_path.append(son_map_path)
        for y in os.listdir(path):
            son_map_relative = y
            map_relative.append(son_map_relative)
    return map_path, map_relative

# 读取图片并处理保存
def read_dispose_img(path):
    map_path ,map_relative= read_img(path)
    imgs = []
    labels = []
    for idx, folder, in enumerate(map_path):
        print("读取图并处理中......"+path_normalization+map_relative[idx])
        normalization(folder,path_normalization+map_relative[idx])


In [6]:
# 数据集地址  最后的斜杆一定不要漏
path = 'D:/QIN/Face-classfication/TestDatabase_1/'
# 处理后的数据集
path_normalization = 'D:/QIN/Face-classfication/TestDatabase_2/'
# 模型保存地址
model_path = './人脸识别/model.ckpt'
# tfrecord文件存放路径
TFRECORD_FILE = "./人脸识别/tfrecords/"

#全局one-hot编码空间
label_binarizer = ""

In [7]:
#     处理照片
read_dispose_img(path)

读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Aaron_Guiel_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Aaron_Patterson_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Aaron_Peirsol_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Aaron_Peirsol_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Aaron_Peirsol_0003.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Aaron_Peirsol_0004.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Aaron_Pena_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Aaron_Sorkin_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Aaron_Sorkin_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Aaron_Tippin_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Abbas_Kiarostami_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Abba_Eban_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_2/Abdel_Aziz_Al-Hakim_0001.jpg
